Este _script_ lee un PDF definido y extrae todas las vacantes (que están en tablas de 14 a 16 columnas).  
Los cursos, tiempos máximos y mínimos, fecha de cobertura y otros campos que no están definidos para todas las vacantes, el _script_ genera una lista de posibles valores, ya que la estructura del PDF del BOD no crea filas en las tablas, sólo una columna.

Algunas tablas presentan una estructura diferente y se tratan por separado, no presenta la EFUN de estas.

In [1]:
import pandas as pd
import numpy as np
import tabula
import math
import os

In [2]:
file_pdf = 'D:\jaume\Datasets\BOD\BOD_20210428_81_destinos_LD.pdf'
read_pdf = tabula.read_pdf(file_pdf, pages = 'all', silent = True)

In [3]:
sel_tables = []                         #creará una lista (sel_tables) con los índices a mantener basado en el número de columnas de la tabla (14-16) y el número de filas (2)
oth_tables = []                         #creará una lista (sel_tables) con los índices a mantener basado en el número de columnas de la tabla (14-16) y el más de 2 filas (funcionan diferente)
for i in range(len(read_pdf)):
    if np.logical_and(read_pdf[i].shape[0] == 2, np.logical_and(read_pdf[i].shape[1] >= 14, read_pdf[i].shape[1] <= 16)):         #seleccionamos sólo los df que contienen de 14 a 16 columnas y 2 filas
        sel_tables.append(i)
        
    elif np.logical_and(read_pdf[i].shape[0] > 2, np.logical_and(read_pdf[i].shape[1] >= 14, read_pdf[i].shape[1] <= 16)):         #seleccionamos sólo los df que contienen de 14 a 16 columnas y 2 filas
        oth_tables.append(i)    
  
    else:
        continue

In [4]:
df_list = [read_pdf[index] for index in sel_tables]    #conservará los elementos de read_tables seleccionados en sel_tables
oth_list = [read_pdf[index] for index in oth_tables]    #conservará los elementos de read_tables seleccionados en sel_tables

In [5]:
df_clean = []
errors = []

In [6]:
#código para los df de shape (2, 14-16) almacenados en df_list

for i in range(len(df_list)):
    df = df_list[i]                                   #iteramos los df seleccionados en df_list
    col_list =[]                                      #instanciamos la lista de columnas vacía que se poblará con los 'split' de cada columna del df de esta iteración
    string = [np.nan] * df.shape[1]
    col = [np.nan] * df.shape[1]
    if type(df.iloc[1, 0]) == str:                    #n_vacs define el número de vacantes que se esperan (num de vacantes separadas por \r en la 1a col del DF)
        n_vacs = len(df.iloc[1, 0].split('\r'))
    else:
        n_vacs = 1

    for j in range(df.shape[1]):                      #iteramos cada columna (str separado por '\r') del df para convertirlo en una lista de valores de la columna

        string[j] = df.iloc[1, j]

        if type(string[j]) == str:

            col[j] = string[j].split('\r')            #columna resultante de la separación de 'string'

            if len(col[j]) == n_vacs:
                col_list.append(col[j])               #si la columna tiene exactamente el mismo número de registros que el número de vacantes del df, adjuntamos la columna

            elif math.ceil(len(col[j])/2) == n_vacs:
                col_list.append(col[j][0::2])         #si la columna tiene el doble (redondeado) de registros que el número de vacantes del df, adjuntamos la columna cada dos espacios

            else:                                     #si no tiene el mismo número, lanzamos la lógica para colocar todos los valores posibles en ese registro y marcamos el error
                errors.append('DF-' + str(i) + ' Elementos no definidos en todas las vacantes. Ver opciones.')
                unique = set(col[j])
                values = list(unique)
                values.append(np.nan)
                col_list.append([values] * n_vacs)
        else:
            col_list.append([np.nan] * n_vacs)
            
    col_map = {}
    if len(col_list) == 14:                             #mapeo según resoluciones de 14 columnas
        col_map = {'n_vac': col_list[0],
                   'uco': col_list[1],
                   'ciu': col_list[2],
                   'pt': col_list[3],
                   'asig': col_list[4],
                   'ta': col_list[5],
                   'empleo': col_list[6],
                   'efun': col_list[7],
                   'cursos': col_list[8],
                   't_max': col_list[9],
                   't_min': col_list[10],
                   'fecha_cob': col_list[11],
                   'csce': col_list[12],
                   'obs': col_list[13]
                  }
            
    elif len(col_list) == 15:                           #mapeo según resoluciones de 15 columnas
        col_map = {'n_vac': col_list[0],
                   'uco': col_list[1],
                   'ciu': col_list[2],
                   'pt': col_list[3],
                   'asig': col_list[4],
                   'ta': col_list[5],
                   'empleo': col_list[6],
                   'efun': col_list[7],
                   'cursos': col_list[8],
                   't_max': col_list[9],
                   't_min': col_list[10],
                   'fecha_cob': col_list[11],
                   'csce': col_list[12],
                   'cod_cm': col_list[13],
                   'obs': col_list[14]
                  }
            
    elif len(col_list) == 16:                           #mapeo según resoluciones de 16 columnas
        col_map = {'n_vac': col_list[0],
                   'uco': col_list[1],
                   'ciu': col_list[2],
                   'pt': col_list[3],
                   'asig': col_list[4],
                   'ta': col_list[5],
                   'ejercito': col_list[6],
                   'cuerpo_esc': col_list[7],
                   'empleo': col_list[8],
                   'efun': col_list[9],
                   'cursos': col_list[10],
                   't_max': col_list[11],
                   't_min': col_list[12],
                   'nivel': col_list[13],   #comprobar que esto es así siempre.
                   'csce': col_list[14],
                   'obs': col_list[15]
                  }

    else:
        errors.append('Oh Mama!')

        
    try:
        df_clean.append(pd.DataFrame(col_map))

    except:
        errors.append('DF-' + str(i) + ' threw an error in DataFrame')

        

In [7]:
str(len(df_clean)) + ' clean df,s in the list, ready to pack'

'4 clean df,s in the list, ready to pack'

In [17]:
#código para los df de shape (2+, 14-16) almacenados en oth_list
oth_clean = []

for oth_df in oth_list:
    
    oth_df = oth_df.iloc[1::2]

    try:
        vac_uco = oth_df.iloc[:, 0].str.split(n = 1, expand = True)
        n_vac = vac_uco.iloc[:, 0]
        uco = vac_uco.iloc[:, 1]
    except:
        continue

    clas_ta = oth_df['CLAS.'].str.split(n = 1, expand = True)
    asig = clas_ta.iloc[:, 0]
    ta = clas_ta.iloc[:, 1]

    obs = oth_df.iloc[:, -1]
    
    pt_ind = 0
    ciu_ind = 0
    cm_ind = 0
    csce_ind = 0
    empleo_ind = 0

    for i in range(oth_df.shape[1]):

        try:
            if round(oth_df.iloc[:, i].str.len().mean()) == 8:
                ciu_ind = i
        except: pass

        try:    
            if oth_df.iloc[:, i].str.count('/').sum() == len(oth_df.iloc[:, i]):
                pt_ind = i
        except: pass

        try:
            if oth_df.iloc[:, i].str.len().mean() == 4:
                cm_ind = i
        except: pass

        try:
            if oth_df.iloc[:, i].str.len().mean() == 6:
                csce_ind = i
        except: pass

        try:    
            if oth_df.iloc[:, i].str.contains('SDO|CBO|CBO 1º|CBMY|SGTO|SGTO 1º|BG|STTE|SBMY|ALF|TTE|CAP|CTE|TCOL|COL').sum() == len(oth_df.iloc[:, i]):
                empleo_ind = i
        except: pass

    col_map = {'n_vac': n_vac,
           'uco': uco,
           'ciu': oth_df.iloc[:, ciu_ind],
           'pt': oth_df.iloc[:, pt_ind],
           'asig': asig,
           'ta': ta,
           'empleo': oth_df.iloc[:, empleo_ind],
#           'efun': col_list[7],
#           'cursos': col_list[8],
#           't_max': col_list[9],
#           't_min': col_list[10],
#           'fecha_cob': col_list[11],
           'csce': oth_df.iloc[:, csce_ind],
           'cod_cm': oth_df.iloc[:, cm_ind],
           'obs': obs
          }
    
    try:
        oth_clean.append(pd.DataFrame(col_map))

    except:
        errors.append('DF_OTH-' + str(i) + ' threw an error in DataFrame')

In [20]:
len(oth_clean)

1

# Pack the df,s!

In [97]:
df_full = pd.concat(df_clean + oth_clean).reset_index(drop = True)      #la solución!!!
df_full

,n_vac,uco,ciu,pt,asig,ta,empleo,efun,cursos,t_max,t_min,fecha_cob,csce,cod_cm,obs
0,03800,BATALLON DE HELICOPTEROS DE EMERGENCIAS II (BE...,50075516,5SA3E/003,CM,C,STTE a BG,"[I.LIG, ARTC, nan]","HC:I, HR:I","[10, nan]","[3, nan]",NaN,"304,34",GM18,"546,656,959"
1,03801,BATALLON DE CG. DE LAS FAMET,50045588,5SA3E/002,CM,C,STTE a BG,"[I.LIG, ARTC, nan]",HP:E,"[10, nan]","[3, nan]",NaN,"374,87",GM18,"546,656,959"
2,03802,BATALLON DE HELICOPTEROS DE EMERGENCIAS II (BE...,50075516,5SA3E/004,CM,C,STTE a BG,"[I.LIG, ARTC, nan]",HP:E,"[10, nan]","[3, nan]",NaN,"374,87",GM18,"546,656,959"
3,03803,BATALLON DE HELICOPTEROS DE MANIOBRA VI,50030082,5SA3E/003,CM,C,STTE a BG,"[I.LIG, ARTC, nan]",HP:E,"[10, nan]","[3, nan]",NaN,"374,87",GM18,"546,656,959"
4,03804,RGTO. DE INF. «AMERICA» N.o 66 DE CAZADORES DE...,50038946,5SA19/002,CM,C,STTE,"[I.LIG, ARTC, nan]",TM:E,"[10, nan]","[3, nan]",NaN,"374,87",GM18,959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,03040,JEFATURA DEL MANDO DE APOYO LOGISTICO,57005389,5WA0A/001,CM,C,TCOL,NaN,NaN,NaN,NaN,NaN,"489,98",PM30,"656,960"
727,03041,JEFATURA DEL MANDO DE APOYO LOGISTICO,57005389,5WA05/004,CM,C,TCOL,NaN,NaN,NaN,NaN,NaN,"489,98",PM30,"656,960"
728,03042,COMANDANCIA DE OBRAS N.o 1 MADRID,54630002,5WA27/001,CM,C,TCOL a CTE,NaN,NaN,NaN,NaN,NaN,"374,87",PM30,960
729,03043,COMANDANCIA DE OBRAS N.o 2 SEVILLA,54630005,5WA65/001,CM,C,CTE,NaN,NaN,NaN,NaN,NaN,"338,86",PM34,960


# Pickle-it!

In [98]:
pkl_dir = 'D:/jaume/Jupyter Notebooks/Vacantes Scraper/ScrapedData/'
pkl_name_parts = file_pdf.split('_')
pkl_name = pkl_name_parts[1] + '_' + pkl_name_parts[2] + '.pkl'

df_full.to_pickle(pkl_dir + pkl_name)